# 원하는 숫자 생성하기

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data', one_hot=True)

total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10

X = tf.placeholder(tf.float32, [None, n_input])

# 이전과 다르게 Y 플레이스홀더를 추가한다.
# 이 플레이스홀더는 결괏값 판정용은 아니고, 노이즈와 실제 이미지에 각각에 해당하는
# 숫자를 힌트로 넣어주는 용도로 사용할 것이다.
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

W0731 21:23:52.582346 4655420864 deprecation.py:323] From <ipython-input-1-674dd9a60d7e>:6: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0731 21:23:52.583013 4655420864 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0731 21:23:52.583851 4655420864 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
# tf.layers 사용
# tf.layers를 사용하게 되면 변수를 선언하지 않고 다음과 같이 tf.variable_scope를 이용해
# 스코프를 지정해줄 수 있다.

def generator(noise, labels):
    with tf.variable_scope('generator'):
        # noise 값에 labels 정보를 간단하게 추가
        inputs = tf.concat([noise, labels], 1)
        # 은닉층을 만든다
        hidden = tf.layers.dense(inputs, n_hidden,
                                activation=tf.nn.relu)
        # 출력층을 구성
        output = tf.layers.dense(hidden, n_input,
                                activation=tf.nn.sigmoid)
    return output

In [3]:
# 구분자 신경망을 만든다.
# 구분자는 진짜 이미지를 판별할 때와 가짜 이미지를 판별할 때 똑같은 변수를 사용해야 한다
# scope.reuse_variables 함수를 이용해 이전에 사용한 변수를 재사용하도록 작성

def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()
            
        inputs = tf.concat([inputs, labels], 1)
        
        hidden = tf.layers.dense(inputs, n_hidden,
                                    activation=tf.nn.relu)
            
        output = tf.layers.dense(hidden, 1,
                                activation=None)
            
    return output
    
# 여기서는 출력값에 활성화 함수를 사용하지 않았는데, 앞서와는 다르게 손실값 계산에
# sigmoid_cross_entropy_with_logits 함수를 사용하기 위함

In [4]:
# 노이즈 생성 함수에서 노이즈를 균등 분로로 생성한다.
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])

In [5]:
# 모델 구성
# 생성자를 구성하고 진짜 이미지 데이터와 생성자가 만든 이미지 데이터를 이용하는
# 구분자를 하나씩 만든다.
# 이때 생성자에는 레이블 정보를 추가하여 추후 레이블 정보에 해당하는 이미지를 생성하도록 유도
# 가짜 이미지 구분자를 만들 때는 진짜 이미지 구분자에서 사용한 변수들을 재사용
# (reuse 옵션을 True)
G = generator(Z, Y)
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

W0731 21:23:53.015916 4655420864 deprecation.py:323] From <ipython-input-2-8eef22bee3e7>:11: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0731 21:23:53.018412 4655420864 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
# 손실 함수
# loss_D 구하기
loss_D_real = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(
                logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(
                logits=D_gene, labels=tf.zeros_like(D_gene)))

# loss_D 는 loss_D_real과 loss_D_gene를 합친 것으로, 이 값을 최소화하면
# 구분자를 학습시킬 수 있다.
# 그러면 D_real은 1에 가까워야하고 D_gene는 0에 가까워야 한다.
loss_D = loss_D_real + loss_D_gene

W0731 21:28:36.507053 4655420864 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
# loss_G 구하기
loss_G = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_gene, labels=tf.ones_like(D_gene)))

In [9]:
# tf.get_collection : discriminator와 generator 스코프에서
# 사용된 변수들을 가져온 뒤, 이 변수들을 최적화에 사용할 각각의 손실 함수와 함께
# 최적화 함수에 넣어 학습 모델 구성 마무리
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                          scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                          scope='generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list=vars_G)

In [10]:
# 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

In [11]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D],
                                feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                feed_dict={Y: batch_ys,
                                          Z: noise})
        
    print('Epoch: ', '%04d' %epoch,
         'D loss: {:.4}'.format(loss_val_D),
         'G loss: {:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G,
                          feed_dict={
                              Y: mnist.test.labels[:sample_size],
                              Z: noise
                          })
        
        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))
        
        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            
            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))
        
        plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)),
                    bbox_inches='tight')
        plt.close(fig)
        
print('최적화 완료!')

Epoch:  0000 D loss: 0.01137 G loss: 7.874
Epoch:  0001 D loss: 0.004404 G loss: 7.638
Epoch:  0002 D loss: 0.004919 G loss: 9.336
Epoch:  0003 D loss: 0.0179 G loss: 7.769
Epoch:  0004 D loss: 0.02782 G loss: 9.412
Epoch:  0005 D loss: 0.05208 G loss: 7.898
Epoch:  0006 D loss: 0.0288 G loss: 8.18
Epoch:  0007 D loss: 0.0636 G loss: 7.788
Epoch:  0008 D loss: 0.06857 G loss: 7.346
Epoch:  0009 D loss: 0.0587 G loss: 6.746
Epoch:  0010 D loss: 0.1667 G loss: 6.184
Epoch:  0011 D loss: 0.1408 G loss: 5.699
Epoch:  0012 D loss: 0.2246 G loss: 4.232
Epoch:  0013 D loss: 0.4725 G loss: 3.26
Epoch:  0014 D loss: 0.3795 G loss: 5.587
Epoch:  0015 D loss: 0.4401 G loss: 4.178
Epoch:  0016 D loss: 0.5988 G loss: 3.645
Epoch:  0017 D loss: 0.4572 G loss: 3.888
Epoch:  0018 D loss: 0.5051 G loss: 3.367
Epoch:  0019 D loss: 0.4669 G loss: 4.006
Epoch:  0020 D loss: 0.5697 G loss: 3.337
Epoch:  0021 D loss: 0.756 G loss: 3.012
Epoch:  0022 D loss: 0.5588 G loss: 2.87
Epoch:  0023 D loss: 0.5485 G 